In [1]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import random
from sklearn.feature_extraction.text import TfidfVectorizer
import sklearn
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score
import scipy.sparse
import re
from nltk.corpus import stopwords
import pymorphy2
import nltk
from nltk.stem.snowball import SnowballStemmer
import string
from sklearn.cluster import KMeans
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [17]:
data = pd.read_csv('regulations.csv')
target = pd.read_csv('train_answer.csv')
data = target.set_index("id").merge(data, on="id")

In [18]:
# избавляемся от лишних дат, оставляя только года
data["public_year"] = pd.to_datetime(data["publication_date"]).dt.year

In [19]:
# избавляемся от бесполезных признаков. Они были обработаны и прогнаны отдельно, поэтому в данном ноутбуке сразу же удаляются
del data["problem_addressed"]
del data["act_objectives"]
del data["persons_affected_by_act"]
del data["relations_regulated_by_act"]
del data["likes_num"]
del data["dislikes_num"]
del data["comments_num"]
del data["publication_date"]
del data["act_changes_controlling_activities"]
del data["is_regionally_signigicant"]
del data["act_significance"]
del data["views_num"]
del data["okved_list"]

In [21]:
data = data[data['act_title'].notna()]

In [23]:
data = data[data['public_year'].notna()]

In [24]:
data["public_year"] = data["public_year"].astype(int)

In [34]:
# избавляемся от всех бесполезных знаков, которые могли бы помешать дальнейшей работе с текстом
act = []
for text in data["act_title"]:
    text = text.lower()
    text = text.replace("\n",' ')
    text = text.replace("\t",' ')
    text = text.replace(".",' ')
    text = text.replace("п о с т а н о в л я е т", 'постановляет')
    text = text.replace("\xa0",' ')
    text = text.replace("\u200e",' ')
    chis = re.sub(r'(\<(/?[^>]+)>)', ' ', text)
    chis = re.sub(' ст ', ' ', chis)
    chis = re.sub('[^а-яА-Я ]', '', chis)
    act.append(chis)

In [36]:
unique = []
for i in range(len(act)):
    for j in act[i]:
        if j not in unique: unique.append(j)

In [38]:
# токенизация слов
act_words = []
for i in range(len(act)):
    act_words.append(word_tokenize(act[i], language='russian'))

In [39]:
a = ord('а')
stroka = ''.join([chr(i) for i in range(a,a+6)] + [chr(a+33)] + [chr(i) for i in range(a+6,a+32)])
list_letters = []
for i in stroka:
    list_letters.append(i)

In [40]:
# подключаем русские стоп-слова
checker = stopwords.words('russian')
checker.extend(list_letters)

In [42]:
# фильтруем эти же слова
filtered_str = []
for i in range(len(act_words)):
    filtered_str.append([word for word in act_words[i] if word not in checker])

In [44]:
# производим их нормализацию - приведение к стандартной форме
morph = pymorphy2.MorphAnalyzer()
done_words = []
for i in range(len(filtered_str)):
    done_words.append([morph.parse(word)[0].normal_form for word in filtered_str[i]])
    if i % 20 == 0: print(i)

0
20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480
3500
3520
3540
3560
3580
3600
3620
3640
3660
3680
3700
3720
3740
3760
3780
3800
3820
3840
3860
3880
3900
3920
3940
3960
3980
4000
4020
4040
4060
4080
4100
4120
4140
4160
4180
4200
4

29220
29240
29260
29280
29300
29320
29340
29360
29380
29400
29420
29440
29460
29480
29500
29520
29540
29560
29580
29600
29620
29640
29660
29680
29700
29720
29740
29760
29780
29800
29820
29840
29860
29880
29900
29920
29940
29960
29980
30000
30020
30040
30060
30080
30100
30120
30140
30160
30180
30200
30220
30240
30260
30280
30300
30320
30340
30360
30380
30400
30420
30440
30460
30480
30500
30520
30540
30560
30580
30600
30620
30640
30660
30680
30700
30720
30740
30760
30780
30800
30820
30840
30860
30880
30900
30920
30940
30960
30980
31000
31020
31040
31060
31080
31100
31120
31140
31160
31180
31200
31220
31240
31260
31280
31300
31320
31340
31360
31380
31400
31420
31440
31460
31480
31500
31520
31540
31560
31580
31600
31620
31640
31660
31680
31700
31720
31740
31760
31780
31800
31820
31840
31860
31880
31900
31920
31940
31960
31980
32000
32020
32040
32060
32080
32100
32120
32140
32160
32180
32200
32220
32240
32260
32280
32300
32320
32340
32360
32380
32400
32420
32440
32460
32480
32500
32520
3254

56560
56580
56600
56620
56640
56660
56680
56700
56720
56740
56760
56780
56800
56820
56840
56860
56880
56900
56920
56940
56960
56980
57000
57020
57040
57060
57080
57100
57120
57140
57160
57180
57200
57220
57240
57260
57280
57300
57320
57340
57360
57380
57400
57420
57440
57460
57480
57500
57520
57540
57560
57580
57600
57620
57640
57660
57680
57700
57720
57740
57760
57780
57800
57820
57840
57860
57880
57900
57920
57940
57960
57980
58000
58020
58040
58060
58080
58100
58120
58140
58160
58180
58200
58220
58240
58260
58280
58300
58320
58340
58360
58380
58400
58420
58440
58460
58480
58500
58520
58540
58560
58580
58600
58620
58640
58660
58680
58700
58720
58740
58760
58780
58800
58820
58840
58860
58880
58900
58920
58940
58960
58980
59000
59020
59040
59060
59080
59100
59120
59140
59160
59180
59200
59220
59240
59260
59280
59300
59320
59340
59360
59380
59400
59420
59440
59460
59480
59500
59520
59540
59560
59580
59600
59620
59640
59660
59680
59700
59720
59740
59760
59780
59800
59820
59840
59860
5988

In [45]:
done_words_save = done_words.copy()

In [46]:
helper = []
for i in done_words_save:
    helper.append(' '.join(i))

In [47]:
data["new_txt"] = helper

In [48]:
data["new_txt"].to_csv(index=False)
compression_opts = dict(method='zip',
                        archive_name='out_act.csv')  
data["new_txt"].to_csv('out_act.zip', index=False,
          compression=compression_opts)

In [81]:
# производим векторизацию слов
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfconverter = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7)
X = tfidfconverter.fit_transform(data["act_title"])

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, data["passed"], train_size=0.85, random_state=0)

In [83]:
# обучаем Кэтбуст. Градиентный бустинг в этой задаче оказался самым полезным и способным. Несмотря на то, что 
# занимает куда больше времени, нежели обычная логистическая регрессия
from catboost import CatBoostClassifier

catmodel = CatBoostClassifier(iterations=2000, learning_rate=0.5) 
catmodel.fit(X_train, y_train) 

0:	learn: 0.4517717	total: 113ms	remaining: 9m 25s
1:	learn: 0.4006130	total: 223ms	remaining: 9m 16s
2:	learn: 0.3768033	total: 338ms	remaining: 9m 22s
3:	learn: 0.3703100	total: 446ms	remaining: 9m 17s
4:	learn: 0.3656718	total: 557ms	remaining: 9m 16s
5:	learn: 0.3609766	total: 669ms	remaining: 9m 16s
6:	learn: 0.3573794	total: 785ms	remaining: 9m 20s
7:	learn: 0.3553889	total: 897ms	remaining: 9m 19s
8:	learn: 0.3535377	total: 1.02s	remaining: 9m 23s
9:	learn: 0.3519152	total: 1.13s	remaining: 9m 24s
10:	learn: 0.3507732	total: 1.24s	remaining: 9m 23s
11:	learn: 0.3490871	total: 1.35s	remaining: 9m 22s
12:	learn: 0.3479164	total: 1.46s	remaining: 9m 21s
13:	learn: 0.3462291	total: 1.57s	remaining: 9m 21s
14:	learn: 0.3449602	total: 1.7s	remaining: 9m 24s
15:	learn: 0.3438617	total: 1.81s	remaining: 9m 23s
16:	learn: 0.3430289	total: 1.92s	remaining: 9m 23s
17:	learn: 0.3422400	total: 2.03s	remaining: 9m 22s
18:	learn: 0.3415550	total: 2.14s	remaining: 9m 21s
19:	learn: 0.3408578	to

158:	learn: 0.2778777	total: 18.3s	remaining: 9m 16s
159:	learn: 0.2775140	total: 18.4s	remaining: 9m 16s
160:	learn: 0.2772422	total: 18.5s	remaining: 9m 16s
161:	learn: 0.2768958	total: 18.6s	remaining: 9m 15s
162:	learn: 0.2765387	total: 18.7s	remaining: 9m 15s
163:	learn: 0.2761949	total: 18.8s	remaining: 9m 15s
164:	learn: 0.2761353	total: 18.9s	remaining: 9m 14s
165:	learn: 0.2756877	total: 19s	remaining: 9m 14s
166:	learn: 0.2756142	total: 19.2s	remaining: 9m 14s
167:	learn: 0.2753238	total: 19.3s	remaining: 9m 14s
168:	learn: 0.2751453	total: 19.4s	remaining: 9m 13s
169:	learn: 0.2747570	total: 19.5s	remaining: 9m 13s
170:	learn: 0.2744896	total: 19.6s	remaining: 9m 13s
171:	learn: 0.2741441	total: 19.7s	remaining: 9m 13s
172:	learn: 0.2738170	total: 19.8s	remaining: 9m 12s
173:	learn: 0.2737631	total: 19.9s	remaining: 9m 12s
174:	learn: 0.2733817	total: 20s	remaining: 9m 12s
175:	learn: 0.2730822	total: 20.1s	remaining: 9m 12s
176:	learn: 0.2728773	total: 20.3s	remaining: 9m 1

316:	learn: 0.2455318	total: 35.6s	remaining: 8m 46s
317:	learn: 0.2452656	total: 35.7s	remaining: 8m 46s
318:	learn: 0.2452327	total: 35.9s	remaining: 8m 46s
319:	learn: 0.2452098	total: 36s	remaining: 8m 45s
320:	learn: 0.2449653	total: 36.1s	remaining: 8m 45s
321:	learn: 0.2447796	total: 36.2s	remaining: 8m 45s
322:	learn: 0.2445450	total: 36.3s	remaining: 8m 45s
323:	learn: 0.2444461	total: 36.4s	remaining: 8m 45s
324:	learn: 0.2444232	total: 36.5s	remaining: 8m 45s
325:	learn: 0.2442563	total: 36.6s	remaining: 8m 45s
326:	learn: 0.2440337	total: 36.8s	remaining: 8m 45s
327:	learn: 0.2437763	total: 36.9s	remaining: 8m 45s
328:	learn: 0.2435946	total: 37s	remaining: 8m 45s
329:	learn: 0.2433976	total: 37.1s	remaining: 8m 44s
330:	learn: 0.2431750	total: 37.2s	remaining: 8m 44s
331:	learn: 0.2429791	total: 37.3s	remaining: 8m 44s
332:	learn: 0.2429520	total: 37.4s	remaining: 8m 44s
333:	learn: 0.2426616	total: 37.5s	remaining: 8m 44s
334:	learn: 0.2425041	total: 37.6s	remaining: 8m 4

472:	learn: 0.2166659	total: 53.1s	remaining: 8m 27s
473:	learn: 0.2164275	total: 53.2s	remaining: 8m 28s
474:	learn: 0.2164085	total: 53.3s	remaining: 8m 27s
475:	learn: 0.2161762	total: 53.4s	remaining: 8m 27s
476:	learn: 0.2161245	total: 53.6s	remaining: 8m 27s
477:	learn: 0.2159596	total: 53.7s	remaining: 8m 27s
478:	learn: 0.2157115	total: 53.8s	remaining: 8m 27s
479:	learn: 0.2156490	total: 53.9s	remaining: 8m 27s
480:	learn: 0.2154851	total: 54s	remaining: 8m 27s
481:	learn: 0.2153145	total: 54.1s	remaining: 8m 27s
482:	learn: 0.2151708	total: 54.3s	remaining: 8m 27s
483:	learn: 0.2148815	total: 54.4s	remaining: 8m 27s
484:	learn: 0.2148349	total: 54.5s	remaining: 8m 27s
485:	learn: 0.2146307	total: 54.6s	remaining: 8m 27s
486:	learn: 0.2144338	total: 54.7s	remaining: 8m 27s
487:	learn: 0.2142588	total: 54.8s	remaining: 8m 26s
488:	learn: 0.2140890	total: 54.9s	remaining: 8m 26s
489:	learn: 0.2138484	total: 55.1s	remaining: 8m 26s
490:	learn: 0.2136417	total: 55.2s	remaining: 8m

628:	learn: 0.1946445	total: 1m 11s	remaining: 8m 16s
629:	learn: 0.1946298	total: 1m 11s	remaining: 8m 16s
630:	learn: 0.1945042	total: 1m 11s	remaining: 8m 16s
631:	learn: 0.1943635	total: 1m 11s	remaining: 8m 16s
632:	learn: 0.1942283	total: 1m 11s	remaining: 8m 15s
633:	learn: 0.1942152	total: 1m 11s	remaining: 8m 15s
634:	learn: 0.1940996	total: 1m 12s	remaining: 8m 15s
635:	learn: 0.1940861	total: 1m 12s	remaining: 8m 15s
636:	learn: 0.1939521	total: 1m 12s	remaining: 8m 15s
637:	learn: 0.1938727	total: 1m 12s	remaining: 8m 15s
638:	learn: 0.1937099	total: 1m 12s	remaining: 8m 15s
639:	learn: 0.1936554	total: 1m 12s	remaining: 8m 15s
640:	learn: 0.1935045	total: 1m 12s	remaining: 8m 14s
641:	learn: 0.1933900	total: 1m 12s	remaining: 8m 14s
642:	learn: 0.1932616	total: 1m 13s	remaining: 8m 14s
643:	learn: 0.1931182	total: 1m 13s	remaining: 8m 14s
644:	learn: 0.1929091	total: 1m 13s	remaining: 8m 14s
645:	learn: 0.1927471	total: 1m 13s	remaining: 8m 14s
646:	learn: 0.1925316	total:

...

4259:	learn: 0.0743057	total: 8m 2s	remaining: 1m 23s
4260:	learn: 0.0742912	total: 8m 2s	remaining: 1m 23s
4261:	learn: 0.0742901	total: 8m 3s	remaining: 1m 23s
4262:	learn: 0.0742827	total: 8m 3s	remaining: 1m 23s
4263:	learn: 0.0742731	total: 8m 3s	remaining: 1m 23s
4264:	learn: 0.0742684	total: 8m 3s	remaining: 1m 23s
4265:	learn: 0.0742556	total: 8m 3s	remaining: 1m 23s
4266:	learn: 0.0742430	total: 8m 3s	remaining: 1m 23s
4267:	learn: 0.0742403	total: 8m 3s	remaining: 1m 22s
4268:	learn: 0.0742362	total: 8m 3s	remaining: 1m 22s
4269:	learn: 0.0742303	total: 8m 4s	remaining: 1m 22s
4270:	learn: 0.0742195	total: 8m 4s	remaining: 1m 22s
4271:	learn: 0.0742194	total: 8m 4s	remaining: 1m 22s
4272:	learn: 0.0742056	total: 8m 4s	remaining: 1m 22s
4273:	learn: 0.0742022	total: 8m 4s	remaining: 1m 22s
4274:	learn: 0.0741989	total: 8m 4s	remaining: 1m 22s
4275:	learn: 0.0741913	total: 8m 4s	remaining: 1m 22s
4276:	learn: 0.0741912	total: 8m 4s	remaining: 1m 21s
4277:	learn: 0.0741868	total

4411:	learn: 0.0732985	total: 8m 19s	remaining: 1m 6s
4412:	learn: 0.0732913	total: 8m 19s	remaining: 1m 6s
4413:	learn: 0.0732831	total: 8m 20s	remaining: 1m 6s
4414:	learn: 0.0732825	total: 8m 20s	remaining: 1m 6s
4415:	learn: 0.0732677	total: 8m 20s	remaining: 1m 6s
4416:	learn: 0.0732619	total: 8m 20s	remaining: 1m 6s
4417:	learn: 0.0732565	total: 8m 20s	remaining: 1m 5s
4418:	learn: 0.0732501	total: 8m 20s	remaining: 1m 5s
4419:	learn: 0.0732500	total: 8m 20s	remaining: 1m 5s
4420:	learn: 0.0732408	total: 8m 20s	remaining: 1m 5s
4421:	learn: 0.0732264	total: 8m 20s	remaining: 1m 5s
4422:	learn: 0.0732219	total: 8m 21s	remaining: 1m 5s
4423:	learn: 0.0732119	total: 8m 21s	remaining: 1m 5s
4424:	learn: 0.0732101	total: 8m 21s	remaining: 1m 5s
4425:	learn: 0.0732015	total: 8m 21s	remaining: 1m 5s
4426:	learn: 0.0731939	total: 8m 21s	remaining: 1m 4s
4427:	learn: 0.0731932	total: 8m 21s	remaining: 1m 4s
4428:	learn: 0.0731840	total: 8m 21s	remaining: 1m 4s
4429:	learn: 0.0731785	total

4565:	learn: 0.0723701	total: 8m 36s	remaining: 49.1s
4566:	learn: 0.0723618	total: 8m 36s	remaining: 49s
4567:	learn: 0.0723584	total: 8m 37s	remaining: 48.9s
4568:	learn: 0.0723518	total: 8m 37s	remaining: 48.8s
4569:	learn: 0.0723476	total: 8m 37s	remaining: 48.7s
4570:	learn: 0.0723367	total: 8m 37s	remaining: 48.6s
4571:	learn: 0.0723332	total: 8m 37s	remaining: 48.4s
4572:	learn: 0.0723298	total: 8m 37s	remaining: 48.3s
4573:	learn: 0.0723241	total: 8m 37s	remaining: 48.2s
4574:	learn: 0.0723234	total: 8m 37s	remaining: 48.1s
4575:	learn: 0.0723179	total: 8m 37s	remaining: 48s
4576:	learn: 0.0723098	total: 8m 38s	remaining: 47.9s
4577:	learn: 0.0723046	total: 8m 38s	remaining: 47.8s
4578:	learn: 0.0722962	total: 8m 38s	remaining: 47.6s
4579:	learn: 0.0722920	total: 8m 38s	remaining: 47.5s
4580:	learn: 0.0722890	total: 8m 38s	remaining: 47.4s
4581:	learn: 0.0722781	total: 8m 38s	remaining: 47.3s
4582:	learn: 0.0722724	total: 8m 38s	remaining: 47.2s
4583:	learn: 0.0722666	total: 8m

4719:	learn: 0.0715453	total: 8m 54s	remaining: 31.7s
4720:	learn: 0.0715386	total: 8m 54s	remaining: 31.6s
4721:	learn: 0.0715352	total: 8m 54s	remaining: 31.5s
4722:	learn: 0.0715325	total: 8m 54s	remaining: 31.3s
4723:	learn: 0.0715325	total: 8m 54s	remaining: 31.2s
4724:	learn: 0.0715284	total: 8m 54s	remaining: 31.1s
4725:	learn: 0.0715216	total: 8m 54s	remaining: 31s
4726:	learn: 0.0715189	total: 8m 54s	remaining: 30.9s
4727:	learn: 0.0715164	total: 8m 54s	remaining: 30.8s
4728:	learn: 0.0715145	total: 8m 55s	remaining: 30.7s
4729:	learn: 0.0715118	total: 8m 55s	remaining: 30.5s
4730:	learn: 0.0715028	total: 8m 55s	remaining: 30.4s
4731:	learn: 0.0715005	total: 8m 55s	remaining: 30.3s
4732:	learn: 0.0714942	total: 8m 55s	remaining: 30.2s
4733:	learn: 0.0714864	total: 8m 55s	remaining: 30.1s
4734:	learn: 0.0714825	total: 8m 55s	remaining: 30s
4735:	learn: 0.0714788	total: 8m 55s	remaining: 29.9s
4736:	learn: 0.0714753	total: 8m 55s	remaining: 29.8s
4737:	learn: 0.0714731	total: 8m

4875:	learn: 0.0707108	total: 9m 12s	remaining: 14s
4876:	learn: 0.0707098	total: 9m 12s	remaining: 13.9s
4877:	learn: 0.0707018	total: 9m 12s	remaining: 13.8s
4878:	learn: 0.0706999	total: 9m 12s	remaining: 13.7s
4879:	learn: 0.0706952	total: 9m 12s	remaining: 13.6s
4880:	learn: 0.0706859	total: 9m 12s	remaining: 13.5s
4881:	learn: 0.0706840	total: 9m 12s	remaining: 13.4s
4882:	learn: 0.0706772	total: 9m 12s	remaining: 13.2s
4883:	learn: 0.0706771	total: 9m 13s	remaining: 13.1s
4884:	learn: 0.0706715	total: 9m 13s	remaining: 13s
4885:	learn: 0.0706696	total: 9m 13s	remaining: 12.9s
4886:	learn: 0.0706653	total: 9m 13s	remaining: 12.8s
4887:	learn: 0.0706637	total: 9m 13s	remaining: 12.7s
4888:	learn: 0.0706586	total: 9m 13s	remaining: 12.6s
4889:	learn: 0.0706526	total: 9m 13s	remaining: 12.5s
4890:	learn: 0.0706526	total: 9m 13s	remaining: 12.3s
4891:	learn: 0.0706457	total: 9m 13s	remaining: 12.2s
4892:	learn: 0.0706436	total: 9m 14s	remaining: 12.1s
4893:	learn: 0.0706357	total: 9m

In [84]:
# данное решение, в конце концов, все же оказалось неверным. На конечном разборе все пришли к общему мнению, что обработка
# текстов в данной задаче является бесполезным занятием, хотя надежды все же были. 
# данный код предназначен скорее для демонстрации умений работы со текстовыми данными
predict = catmodel.predict(X_test)
print(f1_score(y_test, predict))

0.40087623220153346


In [124]:
# пример чуть более успешной работы с данными. Здесь мы отказываемся от текстов, а уделяем чуть более внимание
# данным-классам (разница будет отображаться, если Гитхаб сохранит результаты действий, в ином случае строчки ниже не работают 
# и не имеют смысла)
predict = newcatmodel.predict(X_tel)
print(f1_score(y_tel, predict))

0.585189021232522
